# Data Preprocessing: Create DB and Load CSV into SQLite

In [1]:
import sqlite3
import pandas as pd
import os

# Read CSV
csv_file = 'nft_sales.csv'
df = pd.read_csv(csv_file)
df.head()

,Collections,Sales,Buyers,Txns,Owners
0,Axie Infinity,"$4,090,222,023","1,790,587","17,670,824","2,130,467"
1,Bored Ape Yacht Club,"$2,439,754,017","12,052","32,670","6,586"
2,CryptoPunks,"$2,388,467,992","6,076","22,269","3,804"
3,Mutant Ape Yacht Club,"$1,744,822,678","23,768","51,775","13,121"
4,Art Blocks,"$1,310,734,558","33,549","184,470","36,091"


In [2]:
df.info() # Check for missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Collections  250 non-null    object
 1   Sales        250 non-null    object
 2   Buyers       250 non-null    object
 3   Txns         250 non-null    object
 4   Owners       231 non-null    object
dtypes: object(5)
memory usage: 9.9+ KB


### Converting Sales Column to Int64

In [3]:
df['Sales'] = df['Sales'].astype(str).str.replace('$', '') # Remove dollar sign
df['Sales'] = df['Sales'].astype(str).str.replace(',', '')  # Remove commas
df['Sales'] = df['Sales'].replace('nan', 'NaN')  # To convert 'nan' values as string
df['Sales'] = df['Sales'].fillna('NaN').astype('Int64')  # Convert to int

### Converting Buyers Column to Int64

In [4]:
df['Buyers'] = df['Buyers'].astype(str).str.replace(',', '')  # Remove commas
df['Buyers'] = df['Buyers'].replace('nan', 'NaN')  # To convert 'nan' values as string
df['Buyers'] = df['Buyers'].fillna('NaN').astype('Int64')  # Convert to int

### Converting Txns Column to Int64

In [5]:
df['Txns'] = df['Txns'].astype(str).str.replace(',', '')  # Remove commas
df['Txns'] = df['Txns'].replace('nan', 'NaN')  # To convert 'nan' values as string
df['Txns'] = df['Txns'].fillna('NaN').astype('Int64')  # Convert to int
df['Transactions'] = df['Txns']
df = df.drop(columns=['Txns'])

### Converting Owners Column to Int64

In [6]:
df['Owners'] = df['Owners'].astype(str).str.replace(',', '')  # Remove commas
df['Owners'] = df['Owners'].replace({'nan': None, 'NaN': None, '' : None})  # Handle NaNs properly
df['Owners'] = df['Owners'].astype('Int64')  # Convert to nullable integer type


In [7]:
df.info() # Check for missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Collections   250 non-null    object
 1   Sales         250 non-null    Int64 
 2   Buyers        250 non-null    Int64 
 3   Owners        231 non-null    Int64 
 4   Transactions  250 non-null    Int64 
dtypes: Int64(4), object(1)
memory usage: 10.9+ KB


In [8]:
db_path = os.path.abspath('../database/nft.db')
conn = sqlite3.connect(db_path)
df.to_sql('data', conn, if_exists='replace', index=False)
print("Database saved at:", db_path)
print("Tables:", conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())
print("Data in table 'data':")
print(pd.read_sql_query("SELECT * FROM data LIMIT 5;", conn))

conn.close()

Database saved at: c:\Users\abhin\OneDrive\Desktop\Python Group 6\database\nft.db
Tables: [('data',)]
Data in table 'data':
             Collections       Sales   Buyers   Owners  Transactions
0          Axie Infinity  4090222023  1790587  2130467      17670824
1   Bored Ape Yacht Club  2439754017    12052     6586         32670
2            CryptoPunks  2388467992     6076     3804         22269
3  Mutant Ape Yacht Club  1744822678    23768    13121         51775
4             Art Blocks  1310734558    33549    36091        184470
